In [1]:
import pandas as pd

data = pd.read_csv("clean_weather.csv", index_col=0)

In [2]:
data = data.ffill()

In [195]:
train = data[["tmax", "tmin", "rain"]].to_numpy()[:-1000,:]
target = data[["tmax_tomorrow"]].to_numpy()[:-1000,:]

test = data[["tmax", "tmin", "rain"]].to_numpy()[-1000:,:]
test_target = data[["tmax_tomorrow"]].to_numpy()[-1000:,:]

In [198]:
def finite_diff(func, x):
    eps = 1e-7
    grad = []
    for i in range(len(x)):
        row = np.zeros((1,len(x)))
        row[0,i] = 1
        grad.append((func(x + eps * row) - func(x - eps * row)) / (2 * eps))
    grad = np.array(grad).reshape(len(x), 1)
    print(grad)
    return grad

In [199]:
# Hidden

#1,3 - input
#1,10 - layer1
#1,1 - layer2

# Weights

#3,10
#10,1

#Bias

#1,10
#1,1

#Gradient
# 1,1

# Back
# Step 1
# Gradient dot activated (1,1)
    # Gradient times hidden layer 1 (1,1) * (1,10) = (1,10)
# Weights times gradient 10,1 times 1,1 T = (1,10)

#Step 2
# Gradient dot activated 1,10 dot 1,10 1,10
    # Gradient times input (1,10) * (1,3) ?
# Weights times gradient (3,10) times (10,3) (3,3)

In [196]:
import numpy as np

In [197]:
def mse_loss(predictions, actuals):
    return ((actuals - predictions) ** 2)

In [267]:
class Layer():
    def __init__(self, in_shape, out_shape, relu=True):
        self.weight = np.random.rand(in_shape, out_shape)
        self.bias = np.ones((1, out_shape))
        self.relu = relu
        self.hidden = None
        self.input = None
    
    def forward(self, x):
        self.input = x
        linear = np.matmul(x, self.weight) + self.bias
        self.hidden = linear
        if self.relu:
            relu = np.maximum(linear, np.zeros(linear.shape))
            return relu
        return linear
    
    def backward(self, gradient, prev, lr):
        if self.relu:
            hidden = np.heaviside(self.hidden, 1)
            grad = np.multiply(gradient, hidden).T
        else:
            grad = gradient
        
        w_grad = np.matmul(grad, prev).T
        b_grad = grad.T
        
        self.weight -= w_grad * lr
        self.bias -= b_grad * lr
        
        return np.matmul(self.weight, grad).T

In [268]:
class Network():
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, x):
        output = np.zeros((x.shape[0],1))
        for i in range(x.shape[0]):
            vals = x[i:(i+1),:].copy()
            for layer in self.layers:
                vals = layer.forward(vals)
            output[i:(i+1),:] = vals
        return output
    
    def backward(self, gradient, lr):
        for i in range(gradient.shape[0]):
            elem = gradient[i:(i+1),:]
            for i in range(len(self.layers)-1, -1, -1):
                current = self.layers[i]
                if i > 0:
                    prev = self.layers[i-1].hidden
                else:
                    prev = self.layers[i].input

                elem = current.backward(elem, prev, lr/gradient.shape[0])

In [269]:
layers = [
    Layer(3, 10, True),
    Layer(10, 1, False)
]

net = Network(layers)

In [270]:
epochs = 150
lr = 1e-6

for epoch in range(epochs):
    predictions = net.forward(train)
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} MSE: {np.mean(mse_loss(predictions, target))}")
    outer_grad = predictions - target
    
    # Calc gradients
    net.backward(outer_grad, lr)

Epoch: 0 MSE: 39711.78277639015
Epoch: 10 MSE: 10424.4095763492
Epoch: 20 MSE: 3024.4657387733973
Epoch: 30 MSE: 922.9919479622802
Epoch: 40 MSE: 299.2291942155198
Epoch: 50 MSE: 110.41707307830721
Epoch: 60 MSE: 52.64752152826111
Epoch: 70 MSE: 34.826859819173066
Epoch: 80 MSE: 29.278622893318786
Epoch: 90 MSE: 27.52787282767429
Epoch: 100 MSE: 26.963427825043848
Epoch: 110 MSE: 26.77511504018547
Epoch: 120 MSE: 26.708968881607255
Epoch: 130 MSE: 26.684044920499705
Epoch: 140 MSE: 26.67383553440184
Epoch: 150 MSE: 26.669284160923006
Epoch: 160 MSE: 26.667101664305857
Epoch: 170 MSE: 26.665996500963075
Epoch: 180 MSE: 26.665416070119388
Epoch: 190 MSE: 26.665104234949727


In [271]:
test_preds = net.forward(test)

In [272]:
np.mean(mse_loss(test_preds, test_target))

30.00568557009493